In [ ]:
from tldraw import MakeReal
from api_key import api_key

custom_prompt =  f"""

In the providied image, you will see how a blender scene should look like.
Please try to generate this scene in Blender.
Use the bpy API to generate for that.
Red text and red arrows will describe motion of the animation.
Don't add the text and arrow itself.
Instead, try to understand the motion that is described.
Animate this motion using keyframes.
in anycase USE keyframes please!
Reply ONLY with python code.  
"""


m = MakeReal(width=650, height = 400, api_key = api_key, prompt = custom_prompt , run_next_cell = True)
m

In [ ]:
import bpy

# Clear existing objects
bpy.ops.object.select_all(action='DESELECT')
bpy.ops.object.select_by_type(type='MESH')
bpy.ops.object.delete()

# Create pendulum ball
bpy.ops.mesh.primitive_uv_sphere_add(radius=1, location=(0, 0, 0))
pendulum_ball = bpy.context.object
pendulum_ball.name = 'PendulumBall'

# Create pendulum rod (using a cylinder)
bpy.ops.mesh.primitive_cylinder_add(radius=0.1, depth=10, location=(0, 0, 5))
pendulum_rod = bpy.context.object
pendulum_rod.name = 'PendulumRod'
pendulum_rod.rotation_euler[0] = 3.14159 / 2  # Rotate to horizontal

# Move the origin of the rod to the top
bpy.context.view_layer.objects.active = pendulum_rod
bpy.ops.object.origin_set(type='ORIGIN_CURSOR', center='MEDIAN')

# Parent the ball to the rod
pendulum_ball.parent = pendulum_rod
pendulum_ball.location = (0, 0, -5)  # Position at the bottom of the rod

# Setting the pivot point for the rod
pivot_empty = bpy.data.objects.new("Pivot", None)
bpy.context.collection.objects.link(pivot_empty)
pivot_empty.location = (0, 0, 10)

# Parent rod to the pivot empty
pendulum_rod.parent = pivot_empty

# Set up animation using keyframes
pivot_empty.rotation_mode = 'XYZ'

# Initial position - frame 1
pivot_empty.rotation_euler[2] = 0.5  # radians, adjust as necessary
pivot_empty.keyframe_insert(data_path="rotation_euler", frame=1)

# Middle position - frame 30
pivot_empty.rotation_euler[2] = -0.5  # opposite side
pivot_empty.keyframe_insert(data_path="rotation_euler", frame=30)

# End position - frame 60 (back to start)
pivot_empty.rotation_euler[2] = 0.5
pivot_empty.keyframe_insert(data_path="rotation_euler", frame=60)

# Set scene frames
bpy.context.scene.frame_start = 1
bpy.context.scene.frame_end = 60

# Add cyclic animation modifier
fcurve = pivot_empty.animation_data.action.fcurves.find("rotation_euler", index=2)
modifier = fcurve.modifiers.new('CYCLES')
modifier.mode_before = 'REPEAT'
modifier.mode_after = 'REPEAT'